In [54]:
import pandas as pd
import requests
from collections import defaultdict
import time

input_df = pd.read_csv('addresses.csv', index_col=0)
input_df.head()

,0
0,3DKcHmHdqcUDAX2Yi2sjX9Hauam25Nk5Ja
1,3JNZyt1ocETf9oHscQfx17XNqY3odHAjaR
2,bc1q0xur68lq587lw8aqf96f4qqcmum6sqcpyyy94a
3,3JjtnDXknvY7HWidn5wt5BdZsRdXCGEt8y
4,3J9rQSzvNThPdN5FDbJAD3N8Wdce1KF6Ms


In [55]:
btc_hashes = input_df['0'].values
URL = "https://blockstream.info/api/address/{}/txs/chain"
addr_data = defaultdict(list)
i = 0
for btc_hash in btc_hashes[:50]: # limiting for testing
    last_tx = ''
    while(True):
        if last != '':
            URL+=f'/{last_tx}'
        data = requests.get(URL.format(btc_hash))
        data = data.json()
        addr_data[btc_hash].extend(data)
        if (len(data) < 25):
            break
        last_tx = data[-1]['txid']
    i+=1
    if i % 20 == 0:
        print(f"{i}/{len(btc_hashes)}")
    # time.sleep(1)
print(f"{i}/{len(btc_hashes)}")


20/899
40/899
50/899


In [58]:
addr_data['3DKcHmHdqcUDAX2Yi2sjX9Hauam25Nk5Ja']

[{'txid': 'c5fc5cd22c553207714ac3fa6f51645d47be707b6d51226daaae5b21e0832483',
  'version': 1,
  'locktime': 0,
  'vin': [{'txid': '4a72fc833a5ab16ad45627a3aa499f4b1ab44363a88dda12dadd922cd47b24ed',
    'vout': 1,
    'prevout': {'scriptpubkey': 'a9147f950ba16e5f09cd3206514da708ff9df1eb9fba87',
     'scriptpubkey_asm': 'OP_HASH160 OP_PUSHBYTES_20 7f950ba16e5f09cd3206514da708ff9df1eb9fba OP_EQUAL',
     'scriptpubkey_type': 'p2sh',
     'scriptpubkey_address': '3DKcHmHdqcUDAX2Yi2sjX9Hauam25Nk5Ja',
     'value': 1310546000},
    'scriptsig': '160014a1bb0dfc08d213176dbaf03072c60e90e5285c5e',
    'scriptsig_asm': 'OP_PUSHBYTES_22 0014a1bb0dfc08d213176dbaf03072c60e90e5285c5e',
    'witness': ['304402204803ea7b57f4e91776618cdc3787c932a0cf7b5d43125eb54dd35aec6ccfd80d022038c19200f3e6ac119ac5db1e56d15f7d161ab9646417ccd79d8f8baf6b9737fb01',
     '02aec416909e617f1b798a491cf09f0feedd8efbfbbf3bc78fe00d9fa7bbc0778e'],
    'is_coinbase': False,
    'sequence': 4294967295,
    'inner_redeemscript_asm'